# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.1039735826236234),
 (1, 1.1039735826236234),
 (2, 1.1039735826236234),
 (3, 1.1039735826236234),
 (4, 1.1039735826236234)]

## Inverted index

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('it', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('it', 18)]),
 (1, [('banana', 2), ('is', 18), ('what', 10)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.055129211195594774),
   (None, 0.07925688920363771),
   (None, 0.09064987360443677),
   (None, 0.09148848522085562),
   (None, 0.09682056727627142),
   (None, 0.14491593412427406),
   (None, 0.20297443652491798),
   (None, 0.2480858360914654),
   (None, 0.2537229374018709),
   (None, 0.28950450689780405),
   (None, 0.34440582895563554),
   (None, 0.405737883729819),
   (None, 0.42172209356424184),
   (None, 0.4572520977157871),
   (None, 0.48478063470768107)]),
 (1,
  [(None, 0.5036335391648709),
   (None, 0.5677680272338165),
   (None, 0.5752160094889407),
   (None, 0.6407727432807331),
   (None, 0.7227600328327994),
   (None, 0.7326172273862329),
   (None, 0.7328114209433377),
   (None, 0.749511608312456),
   (None, 0.7581711100809285),
   (None, 0.7840849896627914),
   (None, 0.7918416250152475),
   (None, 0.8645413518693091),
   (None, 0.8801243943357836),
   (None, 0.9319552769638385),
   (None, 0.9323374250868336)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [21]:
def RECORDREADER(input_list):
    return [(1, num) for num in input_list]


def MAP(_, num):
    yield (None, num)


def REDUCE(_, nums):
    yield (None, max(nums))


def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    t = {}
    for (k, v) in iterable:
        t[k] = t.get(k, []) + [v]
    return t.items()


input_list = [3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5]

records = RECORDREADER(input_list)

mapped_values = flatten(MAP(_, num) for _, num in records)

grouped_values = groupbykey(mapped_values)

reduced_values_list = list(flatten(REDUCE(k, v) for k, v in grouped_values))

if reduced_values_list:
    print("Максимальное значение:", reduced_values_list[0][1])
else:
    print("Список пуст")

Максимальное значение: 9


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [22]:
def RECORDREADER(input_list):
    return [(1, num) for num in input_list]

def MAP(_, value):
    yield (None, (value, 1))

def REDUCE(_, values):
    total_sum = 0
    total_count = 0
    for value, count in values:
        total_sum += value
        total_count += count
    average = total_sum / total_count if total_count > 0 else 0
    yield (None, average)

def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    t = {}
    for (k, v) in iterable:
        t[k] = t.get(k, []) + [v]
    return t.items()


input_list = [1, 2, 3, 4, 5, 6]

records = RECORDREADER(input_list)

mapped = flatten(MAP(_, value) for _, value in records)

grouped = groupbykey(mapped)

reduced = flatten(REDUCE(k, v) for k, v in grouped)

average_value = list(reduced)[0][1]
print("Арифметическое среднее:", average_value)

Арифметическое среднее: 3.5


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [23]:
import random

def flatten(lst):
    return list(chain.from_iterable(lst))

def MAP(input_data):
    return (1, input_data)

def groupbykey(items):
    sorted_items = sorted(items, key=lambda x: x[0])
    grouped = {}
    for key, value in sorted_items:
        if key in grouped:
            grouped[key].append(value)
        else:
            grouped[key] = [value]
    return [(k, list(v)) for k, v in grouped.items()]


def REDUCE(key, values):
    total_sum = 0
    total_count = 0
    for value in values:
        total_sum += value
        total_count += 1
    average = total_sum / total_count if total_count > 0 else 0
    return average

def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]


mapped_data = map_output = list(map(lambda x: MAP(x), RECORDREADER(10)))

print(mapped_data)

shuffle_output = groupbykey(mapped_data)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))
print(reduce_output)

[(1, 75), (1, 31), (1, 15), (1, 29), (1, 60), (1, 60), (1, 87), (1, 68), (1, 4), (1, 65)]
[(1, [75, 31, 15, 29, 60, 60, 87, 68, 4, 65])]
[49.4]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [24]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs


d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""

documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
    global maps
    split_size = int(np.ceil(len(documents) / maps))
    for i in range(0, len(documents), split_size):
        yield (
            ("{}:{}".format(docid, lineid), line)
            for docid, document in enumerate(documents[i:i+split_size])
            for lineid, line in enumerate(document.split('\n'))
        )

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, word)

def REDUCE(key:str, value:Iterator[str]):
  yield key

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output


56 key-value pairs were sent over a network.


[(0, ['', 'a', 'it']), (1, ['banana', 'is', 'what'])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [25]:
import random

def MAP(el_list):
    mapped_result = {t: [t] for t in el_list if C(t)}
    return mapped_result.items()

def REDUCE(mapped_items):
    return [key for key, _ in mapped_items]

def C(t):
    return sum(t) > 200
    
def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

record = RECORDREADER(10)

print(record)

part_count = 5
record_partitional = [record[i:i + part_count] for i in range(0, len(record), part_count)]

reduced_output = REDUCE([item for sublist in map(MAP, record_partitional) for item in sublist])
print(reduced_output)

[(87, 97, 35), (6, 69, 69), (22, 100, 57), (65, 62, 53), (30, 56, 8), (82, 37, 5), (89, 90, 73), (53, 24, 92), (51, 3, 74), (31, 45, 89)]
[(87, 97, 35), (89, 90, 73)]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [26]:
import random
from typing import NamedTuple, Iterator, List, Tuple, Set

S: Set[int] = {1, 3, 5, 7, 9, 11, 13, 15}

TupleType = Tuple[int, int, int]

def MAP(t: TupleType) -> Tuple[Tuple[int, ...], Tuple[int, ...]]:
    res = tuple(el for el in t if el in S)
    return (res, res)

def REDUCE(key: Tuple[int, ...], values: Iterator[NamedTuple]) -> Tuple[int, ...]:
    return (key, key)

def RECORDREADER(count: int) -> List[TupleType]:
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for _ in range(count)]

def group_by_key(iterable: List[Tuple[Tuple[int, ...], Tuple[int, ...]]]) -> List[Tuple[Tuple[int, ...], List[Tuple[int, ...]]]]:
    grouped = {}
    for k, v in iterable:
        grouped.setdefault(k, []).append(v)
    return grouped.items()

record = RECORDREADER(5)
map_output = [MAP(x) for x in record]

shuffle_output = list(group_by_key(map_output))

reduce_output = [REDUCE(key, values)[0] for key, values in shuffle_output]

print(record)
print(map_output)
print(shuffle_output)
print(reduce_output)

[(89, 70, 45), (39, 71, 69), (77, 12, 17), (46, 58, 77), (51, 15, 97)]
[((), ()), ((), ()), ((), ()), ((), ()), ((15,), (15,))]
[((), [(), (), (), ()]), ((15,), [(15,)])]
[(), (15,)]


### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [27]:
import random
from typing import List, Tuple, Iterator

TupleType = Tuple[int, int, int]

def MAP(t: TupleType) -> Tuple[TupleType, TupleType]:
    return (t, t)

def REDUCE(key: TupleType, values: Iterator[TupleType]) -> Tuple[TupleType, TupleType]:
    return (key, key)

def RECORDREADER(count: int) -> List[TupleType]:
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for _ in range(count)]

def group_by_key(iterable: List[Tuple[TupleType, TupleType]]) -> List[Tuple[TupleType, List[TupleType]]]:
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return list(t.items())


record = RECORDREADER(3)
map_output = [MAP(x) for x in record]

shuffle_output = list(group_by_key(map_output))

reduce_output = [REDUCE(key, values)[0] for key, values in shuffle_output]

print("Record:\n", record)
print("Map Output:\n", map_output)
print("Shuffle Output:\n", shuffle_output)
print("Reduce Output:\n", reduce_output)

Record:
 [(37, 13, 29), (71, 98, 74), (14, 54, 54)]
Map Output:
 [((37, 13, 29), (37, 13, 29)), ((71, 98, 74), (71, 98, 74)), ((14, 54, 54), (14, 54, 54))]
Shuffle Output:
 [((37, 13, 29), [(37, 13, 29)]), ((71, 98, 74), [(71, 98, 74)]), ((14, 54, 54), [(14, 54, 54)])]
Reduce Output:
 [(37, 13, 29), (71, 98, 74), (14, 54, 54)]


### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [34]:
import random
from typing import Iterator, Tuple

def MAP(t: Tuple[int, int]) -> Tuple[Tuple[int, int], Tuple[int, int]]:
    return (t, t)

def REDUCE(key: Tuple[int, int], values: list) -> Tuple[Tuple[int, int], Tuple[int, int]]:
    if len(values) == 2:
        return (key, key)

def RECORDREADER(count: int) -> list:
    return [(random.randint(0, 3), random.randint(0, 3)) for _ in range(count)]

def group_by_key(iterable: list) -> list:
    grouped = {}
    for key, value in iterable:
        grouped.setdefault(key, []).append(value)
    return grouped.items()


record = RECORDREADER(3)
map_output = [MAP(x) for x in record]

shuffle_output = list(group_by_key(map_output))

reduce_output = [REDUCE(*item) for item in shuffle_output if REDUCE(*item) is not None]

print("Record:\n", record)
print("Map Output:\n", map_output)
print("Shuffle Output:\n", shuffle_output)
print("Reduce Output:\n", [item for item in reduce_output])

Record:
 [(1, 0), (1, 0), (2, 2)]
Map Output:
 [((1, 0), (1, 0)), ((1, 0), (1, 0)), ((2, 2), (2, 2))]
Shuffle Output:
 [((1, 0), [(1, 0), (1, 0)]), ((2, 2), [(2, 2)])]
Reduce Output:
 [((1, 0), (1, 0))]


### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [35]:
import random
from collections import defaultdict
from typing import List, Tuple

REL_R = 0
REL_S = 1

def get_random_tuple() -> Tuple[Tuple[int, int], int]:
    data = (random.randint(0, 3), random.randint(0, 3))
    rel_id = random.choice([REL_R, REL_S])
    return (data, rel_id)

def RECORDREADER(count: int) -> List[Tuple[Tuple[int, int], int]]:
    return [get_random_tuple() for _ in range(count)]

def MAP(t: Tuple[Tuple[int, int], int]) -> Tuple[Tuple[int, int], int]:
    return t

def REDUCE(key: Tuple[int, int], values: List[int]) -> Tuple[Tuple[int, int], Tuple[int, int]]:
    if all(value == REL_R for value in values):
        return (key, key)

def group_by_key(iterable: List[Tuple[Tuple[int, int], int]]) -> dict:
    grouped = defaultdict(list)
    for key, value in iterable:
        grouped[key].append(value)
    return grouped


record = RECORDREADER(5)
map_output = [MAP(x) for x in record]
grouped_output = group_by_key(map_output)
reduce_output = [REDUCE(key, values) for key, values in grouped_output.items() if REDUCE(key, values)]

print(f"Record:\n{record}")
print(f"Map Output:\n{map_output}")
print(f"Shuffle and Group Output:\n{list(grouped_output.items())}")
print(f"Reduce Output:\n{reduce_output}")

Record:
[((1, 2), 0), ((0, 2), 0), ((1, 2), 0), ((0, 0), 0), ((1, 2), 0)]
Map Output:
[((1, 2), 0), ((0, 2), 0), ((1, 2), 0), ((0, 0), 0), ((1, 2), 0)]
Shuffle and Group Output:
[((1, 2), [0, 0, 0]), ((0, 2), [0]), ((0, 0), [0])]
Reduce Output:
[((1, 2), (1, 2)), ((0, 2), (0, 2)), ((0, 0), (0, 0))]


### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [36]:
import random
from typing import List, Tuple

REL_R = 0
REL_S = 1

def get_random_tuple(rel_id: int) -> Tuple[int, Tuple[int, int]]:
    return (rel_id, (random.randint(0, 3), random.randint(0, 3)))

def RECORDREADER(count: int) -> List[Tuple[int, Tuple[int, int]]]:
    return [get_random_tuple(REL_R if i < count // 2 else REL_S) for i in range(count)]

def MAP(t: Tuple[int, Tuple[int, int]]) -> Tuple[int, Tuple[int, int]]:
    rel_id, data = t
    if rel_id == REL_R:
        return (data[1], ('R', data[0]))
    else:
        return (data[0], ('S', data[1]))

def REDUCE(key: int, values: List[Tuple[str, int]]) -> List[Tuple[int, int, int]]:
    result = []
    r_values = [v[1] for v in values if v[0] == 'R']
    s_values = [v[1] for v in values if v[0] == 'S']
    for r in r_values:
        for s in s_values:
            result.append((r, key, s))
    return result

def group_by_key(iterable: List[Tuple[int, Tuple[str, int]]]) -> dict:
    grouped = {}
    for key, value in iterable:
        grouped.setdefault(key, []).append(value)
    return grouped


record = RECORDREADER(7)
map_output = [MAP(x) for x in record]
grouped_output = group_by_key(map_output)
reduce_output = [item for sublist in map(lambda x: REDUCE(*x), grouped_output.items()) for item in sublist]

print(f"Record:\n{record}")
print(f"Map Output:\n{map_output}")
print(f"Grouped Output:\n{list(grouped_output.items())}")
print(f"Reduce Output:\n{reduce_output}")

Record:
[(0, (1, 1)), (0, (0, 2)), (0, (3, 0)), (1, (2, 2)), (1, (0, 3)), (1, (2, 3)), (1, (3, 3))]
Map Output:
[(1, ('R', 1)), (2, ('R', 0)), (0, ('R', 3)), (2, ('S', 2)), (0, ('S', 3)), (2, ('S', 3)), (3, ('S', 3))]
Grouped Output:
[(1, [('R', 1)]), (2, [('R', 0), ('S', 2), ('S', 3)]), (0, [('R', 3), ('S', 3)]), (3, [('S', 3)])]
Reduce Output:
[(0, 2, 2), (0, 2, 3), (3, 0, 3)]


### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [37]:
import random
from typing import List, Tuple, Dict, Any, Iterator

def get_random_tuple() -> Tuple[int, int, int]:
    return (random.randint(0, 3), random.randint(0, 3), random.randint(0, 3))

def RECORDREADER(count: int) -> List[Tuple[int, int, int]]:
    return [get_random_tuple() for _ in range(count)]

def MAP(t: Tuple[int, int, int]) -> Tuple[int, int]:
    return (t[0], t[1])

def theta_aggregate(values: List[int], mode: str) -> Any:
    if mode == 'sum':
        return sum(values)
    elif mode == 'max':
        return max(values)
    elif mode == 'min':
        return min(values)
    elif mode == 'avg':
        return sum(values) / len(values) if values else 0

def REDUCE(key: int, values: List[int], mode: str) -> Tuple[int, Any]:
    aggregated_value = theta_aggregate(values, mode)
    return (key, aggregated_value)

def group_by_key(iterable: List[Tuple[int, int]]) -> Dict[int, List[int]]:
    t = {}
    for k, v in iterable:
        t[k] = t.get(k, []) + [v]
    return t.items()

record = RECORDREADER(10)

print(record)

map_output = [MAP(x) for x in record]

print(map_output)
shuffle_output = group_by_key(map_output)
print(shuffle_output)
aggregations = ['sum', 'max', 'min', 'avg']
for agg in aggregations:
    reduce_output = [REDUCE(key, values, agg) for key, values in shuffle_output]
    print(f"{agg.capitalize()} Reduce Output:\n{reduce_output}")

[(3, 2, 1), (2, 0, 1), (0, 0, 0), (0, 2, 0), (2, 2, 1), (3, 1, 2), (1, 0, 2), (1, 1, 1), (1, 2, 3), (1, 3, 1)]
[(3, 2), (2, 0), (0, 0), (0, 2), (2, 2), (3, 1), (1, 0), (1, 1), (1, 2), (1, 3)]
dict_items([(3, [2, 1]), (2, [0, 2]), (0, [0, 2]), (1, [0, 1, 2, 3])])
Sum Reduce Output:
[(3, 3), (2, 2), (0, 2), (1, 6)]
Max Reduce Output:
[(3, 2), (2, 2), (0, 2), (1, 3)]
Min Reduce Output:
[(3, 1), (2, 0), (0, 0), (1, 0)]
Avg Reduce Output:
[(3, 1.5), (2, 1.0), (0, 1.0), (1, 1.5)]


#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [38]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [39]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):
    yield ((i,k), small_mat[i,j]*w)

def REDUCE(key, values):
  (i, k) = key
  result = sum(values)
  yield ((i,k), result)

Проверьте своё решение

In [40]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [41]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [42]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def RECORDREADER():
    yield from (((0, i, j), value) for i, row in enumerate(small_mat) for j, value in enumerate(row))
    yield from (((1, j, k), value) for j, row in enumerate(big_mat) for k, value in enumerate(row))


def MAP_JOIN(k1, v1):
    mat_num, i, j = k1
    key = j if mat_num == 0 else i
    yield (key, (mat_num, i if mat_num == 0 else j, v1))


def REDUCE_JOIN(key, values):
    for v1 in values:
        if v1[0] == 0:
            for v2 in values:
                if v2[0] == 1:
                    yield ((v1[1], v2[1]), v1[2] * v2[2])

def MAP_MUL(k1, v1):
    yield k1, v1


def REDUCE_MUL(key, values):
    yield (key, sum(values))


def GET_JOINED():
    yield from joined

joined = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
solution = MapReduce(GET_JOINED, MAP_MUL, REDUCE_MUL)
np.allclose(reference_solution, asmatrix(solution)) 

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [43]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs


I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def INPUTFORMAT():
    first_mat = [((0, i, j), value) for i, row in enumerate(small_mat) for j, value in enumerate(row)]
    second_mat = [((1, j, k), value) for j, row in enumerate(big_mat) for k, value in enumerate(row)]
    return [first_mat, second_mat]



def MAP_JOIN(k1, v1):
    mat_num, i, j = k1
    key = j if mat_num == 0 else i
    yield (key, (mat_num, i if mat_num == 0 else j, v1))


def REDUCE_JOIN(key, values):
    for v1 in values:
        if v1[0] == 0:
            for v2 in values:
                if v2[0] == 1:
                    yield ((v1[1], v2[1]), v1[2] * v2[2])


def GET_JOINED():
  yield from (j[1] for j in joined)


def MAP_MUL(k1, v1): #+
    yield k1, v1


def REDUCE_MUL(key, values):
    yield (key, sum(values))

maps = 8
reducers = 4
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = list(map(lambda x: (x[0], list(x[1])), partitioned_output))

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = list(map(lambda x: (x[0], list(x[1])), mul_output))

solution = [v for p in pre_result for v in p[1]]

np.allclose(reference_solution, asmatrix(solution)) # should return true

126 key-value pairs were sent over a network.
240 key-value pairs were sent over a network.


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [44]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs


I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def INPUTFORMAT():
  first_mat = [((0, i, j), small_mat[i,j]) for i in range(small_mat.shape[0]) for j in range(small_mat.shape[1])]

  second_mat = [((1, j, k), big_mat[j,k]) for j in range(big_mat.shape[0]) for k in range(big_mat.shape[1])]

  # Разбиваем первую матрицу на части
  split_size = int(np.ceil(len(first_mat)/maps))
  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]

  # Разбиваем вторую матрицу на части
  split_size = int(np.ceil(len(second_mat)/maps))
  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]




def MAP_JOIN(k1, v1):
    mat_num, i, j = k1
    key = j if mat_num == 0 else i
    yield (key, (mat_num, i if mat_num == 0 else j, v1))


def REDUCE_JOIN(key, values):
    for v1 in values:
        if v1[0] == 0:
            for v2 in values:
                if v2[0] == 1:
                    yield ((v1[1], v2[1]), v1[2] * v2[2])


def GET_JOINED():
  yield from (j[1] for j in joined)


def MAP_MUL(k1, v1): #+
    yield k1, v1


def REDUCE_MUL(key, values):
    yield (key, sum(values))

maps = 8
reducers = 4
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = list(map(lambda x: (x[0], list(x[1])), partitioned_output))

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = list(map(lambda x: (x[0], list(x[1])), mul_output))

solution = [v for p in pre_result for v in p[1]]

np.allclose(reference_solution, asmatrix(solution)) # should return true

126 key-value pairs were sent over a network.
240 key-value pairs were sent over a network.


True